In [980]:
import numpy as np 
import pandas as pd
np.random.seed(42)

In [1104]:
dados = pd.read_csv("dadoss.csv", delimiter=";", decimal=",")
dados = dados[["SAPR4", "ELET6","VIVT3", "MGLU3"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(4, 1)

In [1056]:
def timing(func): 
    import time      
    def wrapper(*args, **kwargs):          
        t = time.clock()          
        res = func(*args, **kwargs)          
        print(func.__name__, time.clock()-t)         
        return res      
    return wrapper 

In [1057]:
def minimize_ipm(rp, e_r, cov, w):
    n =  e_r.shape[0]
    
    #Primeiro Chute
    #w = (np.random.randn(n)).reshape(n, 1)#Carteira
    l = np.array([[1],[1/2]])#Multiplicadores de Lagrange
    
    #Calculando grad(c(x)) 
    dc = np.hstack((np.ones((n,1)), e_r))

    #Criando a Matriz A
    def generate_A(Sigma, dc, n, delta1 = 0, delta2 =0.0):
        a = np.hstack([2*Sigma+delta1*np.eye(len(w)), dc])
        b = np.hstack([dc.T, np.zeros((2,2))-delta2*np.eye(2)])
        return np.vstack((a, b))
    
    #Criando o vetor b.
    def generate_b(Sigma, dc, e_r, l, w, rp):
        a = 2*Sigma@w + dc@l
        b = np.ones((1,n))@w - 1
        c = e_r.T@w - rp
        return -1*np.vstack((a, b, c))
    
    def grad(w, cov, l, dc):
        return (2*cov@w+ dc@l)
    
    A = generate_A(cov, dc, n)
    

    for i in range(100):                                               #while max(np.abs((grad(w,cov, l, dc)))) > eps:
        b = generate_b(cov, dc, e_r, l, w, rp)
        #print("b: ",b, "\n")
        dx = np.linalg.inv(A)@b
        #print("dx: ", dx, "\n")
        dw, dl = dx[0:n][:], dx[n:n+2][:]
        w = w + 0.01*dw
        #print("w: \n", w, "\n")
        l = l + 0.01*dl
        
    return w

In [1236]:
#@timing
def minimize_gen(rp, er, cov, w_anterior):
    n = er.shape[0]
    
    def chromossome(n):
        x = np.random.rand(n)
        return x/sum(x)
    
    def fitness_function(w, e_r=e_r, cov=cov, rp=rp, alpha = 10, beta = 1000):
        w = w.reshape(n, 1)
        return (w.T@cov@w + beta*(w.T@e_r -rp)**2)[0][0]
    
    def select_elite(population):
        i = np.argmin([fitness_function(_) for _ in population])
        return population[i]
    
    def new_population(population):
        new_population = []
        intermediate_population=[]
    
        elite = select_elite(population)
    
        for candidate in population:
            c1, c2 = population[np.random.randint(0, len(population))][:], population[np.random.randint(0, len(population))][:]
            ic = candidate +1*(c1-c2)
        
        for j in range(len(candidate)):
            ic[j] = np.random.choice([elite[j], ic[j]], size = 1, p=[0.9, 0.1])
            
        ic = np.where(ic>=0, ic, 0)
        
        ic = ic/sum(ic)
        
        if fitness_function(ic) <= fitness_function(candidate):
            new_population.append(ic)
        else: 
            new_population.append(candidate)
    
        return new_population
    

    population_size = 100
    population = []
    for i in range(population_size):
        population.append(w_anterior+.001*np.random.randn(len(w_anterior)))
    for i in range(100):
        population = new_population(population)

    return(select_elite(population).reshape(n))

In [1243]:
from scipy.optimize import minimize
def minimize_vol2(target_return, er, cov):
    """
    target_ret -> W
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    return_is_target = {'type': 'eq',
                        'args': (er,),
                        'fun': lambda weights, er: target_return - weights.T@er
    }
    
    results = minimize(lambda weights, cov: weights.T@cov@weights, init_guess,
                       args=(cov,), method="SLSQP",
                       options={"disp":False},
                       constraints=(return_is_target, weights_sum_to_1),
                       bounds=bounds
                      )
    return results.x
    

In [1244]:
def minimize_fodastico(rp, er, cov):
    w = minimize_gen(rp, er, cov).reshape((4, 1))
    w = minimize_ipm(rp, er, cov, w)
    
    return w

In [1239]:
returns = np.linspace(min(e_r), max(e_r), 10)

In [1240]:
carteiras_scipy = np.array([minimize_vol(r, e_r, cov).reshape((4, 1)) for r in returns])

In [1241]:
index = np.argmin(e_r)
w = np.zeros(e_r.shape[0])
w[index] = 1
carteiras_pi =[]

for r in returns:
    carteiras_pi.append(w)
    w = minimize_gen(r, e_r, cov, w_anterior= w)


In [1246]:
dados = pd.read_csv("dados_ajustados.csv", decimal= ",", delimiter =";")
dados = dados[["SPXI", "IBOV"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(2, 1)


returns = np.linspace(min(e_r), max(e_r), 10)
carteiras_pi = np.array([np.round(minimize_aleatorio(r, e_r, cov), 2) for r in returns])
carteiras_scipy = np.array([minimize_vol(r, e_r, cov).reshape((2, 1)) for r in returns])

vol1 = [float(np.sqrt(w.T@cov@w)) for w in carteiras_pi]
dic1 = {"Vol": vol1, "r": returns.reshape(returns.shape[0])}
df1 = pd.DataFrame(dic1)
ax = df1.plot(x="Vol", y="r", marker = "x", color="goldenrod", figsize=(12,6))

vol2 = [float(np.sqrt(w.T@cov@w)) for w in carteiras_scipy]
dic2 = {"Vol": vol2, "r": returns.reshape(returns.shape[0])}
df2 = pd.DataFrame(dic2)
df2.plot(x="Vol", y="r", marker = "x", color="red", ax = ax)

NameError: name 'minimize_aleatorio' is not defined

In [1228]:
df1

,Vol,r
0,0.018739,0.000451
1,0.018734,0.000451
2,0.018729,0.000451
3,0.018729,0.000452
4,0.018732,0.000452
5,0.018701,0.000452
6,0.018729,0.000451
7,0.018738,0.000451
8,0.018727,0.000452
9,0.018726,0.000452


In [1165]:
carteiras_pi

array([[1.78187046e-01, 3.02027236e-01, 5.05124184e-01, 1.46615334e-02],
       [2.45837362e-01, 3.01736667e-01, 3.94033456e-01, 5.83925144e-02],
       [4.11222733e-01, 3.42899048e-02, 4.18120151e-01, 1.36367211e-01],
       [2.98430585e-01, 3.10613764e-05, 4.29925394e-01, 2.71612959e-01],
       [2.86757178e-01, 2.03313224e-01, 3.14602060e-01, 1.95327538e-01],
       [1.21233236e-01, 7.00031079e-03, 5.00787087e-01, 3.70979366e-01],
       [2.03609937e-01, 7.89116585e-02, 3.50598550e-01, 3.66879854e-01],
       [2.07758594e-01, 6.45535194e-02, 3.45044894e-01, 3.82642993e-01],
       [9.94357869e-02, 3.95888126e-02, 3.71127244e-01, 4.89848157e-01],
       [2.07212919e-02, 1.16930752e-01, 2.09909114e-01, 6.52438842e-01]])

In [1166]:
carteiras_scipy

array([[[9.44949918e-08],
        [0.00000000e+00],
        [9.99999906e-01],
        [1.27675648e-15]],

       [[3.18449870e-01],
        [2.88027412e-01],
        [3.19303929e-01],
        [7.42187896e-02]],

       [[2.73959078e-01],
        [2.63068710e-01],
        [2.74401807e-01],
        [1.88570404e-01]],

       [[2.29468287e-01],
        [2.38110009e-01],
        [2.29499685e-01],
        [3.02922019e-01]],

       [[1.84977495e-01],
        [2.13151308e-01],
        [1.84597563e-01],
        [4.17273633e-01]],

       [[1.40486704e-01],
        [1.88192607e-01],
        [1.39695442e-01],
        [5.31625248e-01]],

       [[9.59959125e-02],
        [1.63233905e-01],
        [9.47933196e-02],
        [6.45976862e-01]],

       [[5.15051211e-02],
        [1.38275204e-01],
        [4.98911977e-02],
        [7.60328477e-01]],

       [[7.01432528e-03],
        [1.13316505e-01],
        [4.98907780e-03],
        [8.74680092e-01]],

       [[0.00000000e+00],
        [2.44784970e